# Data wrangling customer data & combine with orders-product dataset 

## Part 1 task 4.9

## 1. Importing libraries and data sets

In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
#creating pathway for dataframes
path = os.path.join(r'/Users/katja/Documents/Instacart Basket Analysis/02 Data')

In [3]:
#import orders_products
ords_prods =pd.read_pickle(os.path.join(path,'Prepared Data', 'orders_products_merged_updated.pkl'))

In [8]:
#import the customer file and compress the numeric values where possible

dtypes = {'Age': 'int8','n_dependants': 'int8', 'income':'int32'}

df_customers = pd.read_csv(os.path.join(path,'Original Data','customers.csv'),dtype=dtypes)

In [9]:
#checking types
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int8  
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int8  
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int32 
dtypes: int32(1), int64(1), int8(2), object(6)
memory usage: 12.2+ MB


## Data Wrangling

In [11]:
#Checking Columns
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [12]:
#checking rows, columns
df_customers.shape

(206209, 10)

In [13]:
#data description
df_customers.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [14]:
#fam_status value_counts
df_customers['fam_status'].value_counts(dropna=False)

fam_status
married                             144906
single                               33962
divorced/widowed                     17640
living with parents and siblings      9701
Name: count, dtype: int64

In [15]:
#date_joined value_counts
df_customers['date_joined'].value_counts(dropna=False)

date_joined
9/17/2018     213
2/10/2018     212
4/1/2019      211
9/21/2019     211
12/19/2017    210
             ... 
9/1/2018      141
1/22/2018     140
11/24/2017    139
7/18/2019     138
8/6/2018      128
Name: count, Length: 1187, dtype: int64

# Updating column type, drop and rename

### Changing column type

In [16]:
# update user_id from int to str
df_customers['user_id'] = df_customers['user_id'].astype('str')
df_customers['date_joined'] = pd.to_datetime(df_customers['date_joined'])

In [17]:
# changing user_id from int64 to string on ords_prods
ords_prods['user_id'] = ords_prods['user_id'].astype('str')

In [18]:
# changing order_id from int64 to string 
ords_prods['order_id'] = ords_prods['order_id'].astype('str')

In [19]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   user_id       206209 non-null  object        
 1   First Name    194950 non-null  object        
 2   Surnam        206209 non-null  object        
 3   Gender        206209 non-null  object        
 4   STATE         206209 non-null  object        
 5   Age           206209 non-null  int8          
 6   date_joined   206209 non-null  datetime64[ns]
 7   n_dependants  206209 non-null  int8          
 8   fam_status    206209 non-null  object        
 9   income        206209 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int8(2), object(6)
memory usage: 12.2+ MB


In [20]:
#code to check for columns that have mixed data type
for col in df_customers.columns.tolist():
    weird = (df_customers[[col]].map(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
    
    if len (df_customers[weird]) > 0:
        print (col)

First Name


In [21]:
# changing first_name to string as its a mixed type
df_customers['First Name'] = df_customers['First Name'].astype('str')

#### Observations:

##### changed user_id (customers df) to string 
##### changed user_id (ords prods df) to string
##### changed order_id (ords prods df) to string
##### First name has mixed types. changed to string

### Update Column names

In [22]:
#rename First Name, surnam, Gender, STATE and Age
df_customers.rename(columns={"First Name":"first_name", "Surnam":"last_name", "Gender":"gender",'STATE':'state', 'fam_status':'martital_status',
                            "Age":"age"}, inplace=True)

df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   user_id          206209 non-null  object        
 1   first_name       206209 non-null  object        
 2   last_name        206209 non-null  object        
 3   gender           206209 non-null  object        
 4   state            206209 non-null  object        
 5   age              206209 non-null  int8          
 6   date_joined      206209 non-null  datetime64[ns]
 7   n_dependants     206209 non-null  int8          
 8   martital_status  206209 non-null  object        
 9   income           206209 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int8(2), object(6)
memory usage: 12.2+ MB


## Consistency Check

### Missing Values

In [25]:
#missing value count
df_customers.isnull().sum()

user_id            0
first_name         0
last_name          0
gender             0
state              0
age                0
date_joined        0
n_dependants       0
martital_status    0
income             0
dtype: int64

#### Observations:

##### no missing values

### Duplicates

In [66]:
#creating subset for duplicates and checking for duplicates
df_dups = df_customers[df_customers.duplicated()]
df_dups.shape

(0, 7)

#### Observations:

##### no duplicates

### Dropping columns

In [30]:
# dropping colmn surname/ last_name for privacy reason 
df_customers = df_customers.drop(columns =['last_name'])

In [31]:
# dropping colmn first_name for privacy reason 
df_customers = df_customers.drop(columns =['first_name'])

### Consistency check for single columns

#### Gender

In [32]:
#check value types and counts of gender
df_customers['gender'].value_counts(dropna=False)

gender
Male      104067
Female    102142
Name: count, dtype: int64

#### Observations:

##### no changes required

#### State

In [34]:
df_customers['state'].nunique()

51

In [35]:
# list states to check for duplicates and error names.
df_customers['state'].value_counts(dropna=False)

state
Florida                 4044
Colorado                4044
Illinois                4044
Alabama                 4044
District of Columbia    4044
Hawaii                  4044
Arizona                 4044
Connecticut             4044
California              4044
Indiana                 4044
Arkansas                4044
Alaska                  4044
Delaware                4044
Iowa                    4044
Idaho                   4044
Georgia                 4044
Wyoming                 4043
Mississippi             4043
Oklahoma                4043
Utah                    4043
New Hampshire           4043
Kentucky                4043
Maryland                4043
Rhode Island            4043
Massachusetts           4043
Michigan                4043
New Jersey              4043
Kansas                  4043
South Dakota            4043
Minnesota               4043
Tennessee               4043
New York                4043
Washington              4043
Louisiana               4043
Montana 

#### Observations:

##### 51 states 
##### customers almost equally apread out over the states

#### Age

In [37]:
df_customers['age'].describe()

count    206209.000000
mean         49.501646
std          18.480962
min          18.000000
25%          33.000000
50%          49.000000
75%          66.000000
max          81.000000
Name: age, dtype: float64

In [38]:
#checking for any null values or odd ages
df_customers['age'].value_counts()

age
19    3329
55    3317
51    3317
56    3306
32    3305
      ... 
65    3145
25    3127
66    3114
50    3102
36    3101
Name: count, Length: 64, dtype: int64

#### Observations:

##### No missing values 
##### No null values or odd ages

#### Date joined

In [41]:
#min value
df_customers['date_joined'].min()

Timestamp('2017-01-01 00:00:00')

In [42]:
#max value
df_customers['date_joined'].max()

Timestamp('2020-04-01 00:00:00')

#### Observations:

##### Date joined are from 2017-01-01 till 2020-04-01

#### n-dependants

In [43]:
# value counts of n_dependants
df_customers['n_dependants'].value_counts(dropna=False)

n_dependants
0    51602
3    51594
1    51531
2    51482
Name: count, dtype: int64

#### Observations:

##### No missing or inconsistent in n_dependants

#### marital_status

In [45]:
#rename martital_status to marital_status
df_customers.rename(columns={"martital_status":"marital_status"}, inplace=True)

df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   user_id         206209 non-null  object        
 1   gender          206209 non-null  object        
 2   state           206209 non-null  object        
 3   age             206209 non-null  int8          
 4   date_joined     206209 non-null  datetime64[ns]
 5   n_dependants    206209 non-null  int8          
 6   marital_status  206209 non-null  object        
 7   income          206209 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int8(2), object(4)
memory usage: 9.0+ MB


In [46]:
df_customers['marital_status'].value_counts(dropna=False)

marital_status
married                             144906
single                               33962
divorced/widowed                     17640
living with parents and siblings      9701
Name: count, dtype: int64

#### Observations:

##### The majority clients/customer are married. 

#### income

In [47]:
df_customers['income'].describe()

count    206209.000000
mean      94632.852548
std       42473.786988
min       25903.000000
25%       59874.000000
50%       93547.000000
75%      124244.000000
max      593901.000000
Name: income, dtype: float64

In [48]:
df_customers['income'].median()

93547.0

#### Observations:

#####  Income's distribution is right skewed where by mean and median are between 93,547 - 94,632

## Merging customers.csv with orders_products_merged_updated

In [49]:
# checking for _merge column
ords_prods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 22 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                object 
 1   user_id                 object 
 2   clients_order_number    int64  
 3   orders_day_of_week      int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   product_id              int64  
 7   add_to_cart_order       int64  
 8   reordered               int64  
 9   product_name            object 
 10  aisle_id                int64  
 11  department_id           int64  
 12  prices                  float64
 13  price_range_loc         object 
 14  busiest_days            object 
 15  busiest_period_of_day   object 
 16  max_order               int64  
 17  loyalty_flag            object 
 18  mean_spend              float64
 19  spending_flag           object 
 20  median_order_frequency  float64
 21  order_frequency_flag    objec

In [52]:
#Change types for ords_prods data set to save GB

ords_prods['clients_order_number']=ords_prods['clients_order_number'].astype('int8')
ords_prods['orders_day_of_week']=ords_prods['orders_day_of_week'].astype('int8')
ords_prods['order_hour_of_day']=ords_prods['order_hour_of_day'].astype('int8')
ords_prods['days_since_prior_order']=ords_prods['days_since_prior_order'].astype('float16')
ords_prods['product_id']=ords_prods['product_id'].astype('int32')
ords_prods['add_to_cart_order'] = ords_prods['add_to_cart_order'].astype('int32')
ords_prods['reordered']=ords_prods['reordered'].astype('int8')
ords_prods['aisle_id']=ords_prods['aisle_id'].astype('int8')
ords_prods['department_id']=ords_prods['department_id'].astype('int8')
ords_prods['prices']=ords_prods['prices'].astype('float16')
ords_prods['max_order']=ords_prods['max_order'].astype('int8')
ords_prods['mean_spend'] = ords_prods['mean_spend'].astype('float32')
ords_prods['median_order_frequency']=ords_prods['median_order_frequency'].astype('float32')

/Users/katja/anaconda3/lib/python3.11/site-packages/pandas/core/dtypes/astype.py:134: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


In [53]:
ords_prods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 22 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                object 
 1   user_id                 object 
 2   clients_order_number    int8   
 3   orders_day_of_week      int8   
 4   order_hour_of_day       int8   
 5   days_since_prior_order  float16
 6   product_id              int32  
 7   add_to_cart_order       int32  
 8   reordered               int8   
 9   product_name            object 
 10  aisle_id                int8   
 11  department_id           int8   
 12  prices                  float16
 13  price_range_loc         object 
 14  busiest_days            object 
 15  busiest_period_of_day   object 
 16  max_order               int8   
 17  loyalty_flag            object 
 18  mean_spend              float32
 19  spending_flag           object 
 20  median_order_frequency  float32
 21  order_frequency_flag    objec

In [54]:
ords_prods.head()

,order_id,user_id,clients_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,prices,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_spend,spending_flag,median_order_frequency,order_frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,9.0,Mid-range product,regularly busy days,least orders,10,New customer,6.367796,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,9.0,Mid-range product,least busy days,least orders,10,New customer,6.367796,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,9.0,Mid-range product,least busy days,most orders,10,New customer,6.367796,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,9.0,Mid-range product,least busy days,least orders,10,New customer,6.367796,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,9.0,Mid-range product,least busy days,most orders,10,New customer,6.367796,Low spender,20.5,Non-frequent customer


In [55]:
# checking for _merge column & ensuring the key columns are same data type
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   user_id         206209 non-null  object        
 1   gender          206209 non-null  object        
 2   state           206209 non-null  object        
 3   age             206209 non-null  int8          
 4   date_joined     206209 non-null  datetime64[ns]
 5   n_dependants    206209 non-null  int8          
 6   marital_status  206209 non-null  object        
 7   income          206209 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int8(2), object(4)
memory usage: 9.0+ MB


In [68]:
#merging tables on orders_prods using 'inner' statement
df_orders_products_customers = ords_prods.merge(df_customers, on=['user_id'], indicator=True)

In [58]:
#checking columns
df_orders_products_customers.head()

,order_id,user_id,clients_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,median_order_frequency,order_frequency_flag,gender,state,age,date_joined,n_dependants,marital_status,income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,20.5,Non-frequent customer,Female,Alabama,31,2019-02-17,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,20.5,Non-frequent customer,Female,Alabama,31,2019-02-17,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,20.5,Non-frequent customer,Female,Alabama,31,2019-02-17,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,20.5,Non-frequent customer,Female,Alabama,31,2019-02-17,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,20.5,Non-frequent customer,Female,Alabama,31,2019-02-17,3,married,40423,both


In [59]:
df_orders_products_customers['_merge'].value_counts(dropna=False)

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

In [60]:
#drop _merge column
df_orders_products_customers = df_orders_products_customers.drop(columns=['_merge'])

In [61]:
df_orders_products_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 29 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   order_id                object        
 1   user_id                 object        
 2   clients_order_number    int8          
 3   orders_day_of_week      int8          
 4   order_hour_of_day       int8          
 5   days_since_prior_order  float16       
 6   product_id              int32         
 7   add_to_cart_order       int32         
 8   reordered               int8          
 9   product_name            object        
 10  aisle_id                int8          
 11  department_id           int8          
 12  prices                  float16       
 13  price_range_loc         object        
 14  busiest_days            object        
 15  busiest_period_of_day   object        
 16  max_order               int8          
 17  loyalty_flag            object        
 18  

## Export merged dataset

In [63]:
#Export df_ords_prods_merge to PKL
df_orders_products_customers.to_pickle(os.path.join(path,'Prepared Data', 'orders_products_customers_updated.pkl'))

In [73]:
#loading garbage collector
import gc

In [75]:
#delete df to save space 

del df_orders_products_customers
del df_dups

gc.collect()

NameError: name 'df_orders_products_customers' is not defined